### setup

In [1]:
%load_ext lab_black

In [2]:
import requests
import urllib.request
import numpy as np
import datetime
import random

In [3]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [6]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [38]:
scope = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "jupyter-integration-credentials.json", scope
)
gc = gspread.authorize(credentials)

### read in data

In [14]:
df_us = pd.read_excel("data/raw/lowy_slim.xlsx", sheet_name="US")

In [17]:
df_china = pd.read_excel("data/raw/lowy_slim.xlsx", sheet_name="China", skiprows=2)

In [19]:
china_total = df_china[["Row Labels", "Grand Total"]]

In [21]:
china_total.columns = ["country", "china"]

In [23]:
df_us.columns = df_us.columns.str.lower()

In [25]:
us_total = df_us[["country", "grand total"]]

In [26]:
us_total.columns = ["country", "us"]

In [29]:
us_china = us_total.merge(china_total, on="country")

In [34]:
us_china[us_china["country"] != "Grand Total"]

,country,us,china
0,Cook Islands,72680,6.309566e+07
1,Fiji,24253128,3.073780e+08
2,"Micronesia, Federated States of",868669977,6.411541e+07
3,Niue,63382,2.000000e+04
4,"Oceania, regional",80034401,2.000000e+05
5,Papua New Guinea,73120596,7.123071e+08
6,Samoa,13492725,3.542694e+08
7,Solomon Islands,29052925,3.868800e+04
8,Tonga,14374286,2.176662e+08
9,Vanuatu,70973917,2.262781e+08


In [35]:
# dw.create_chart("us vs china spending", chart_type="d3-bars-split", data=us_china)
range_chart = "UceVO"
bullets = "Rpiod"
split = "nDBSC"
grouped = "PVPo7"

In [36]:
dw.add_data(chart_id=range_chart, data=us_china[us_china["country"] != "Grand Total"])

<Response [204]>

In [103]:
dw.add_data(chart_id=bullets, data=us_china[us_china["country"] != "Grand Total"])
dw.add_data(chart_id=split, data=us_china[us_china["country"] != "Grand Total"])
dw.add_data(chart_id=grouped, data=us_china[us_china["country"] != "Grand Total"])

<Response [204]>

### bring in flag codes

In [39]:
# Opening the worksheet by using Worksheet ID

spreadsheet_key = "1IUYlGHQ6l87G6_LVeNlfsdI02cTLvll9mNw0-HVJ4ZI"
workbook = gc.open_by_key(spreadsheet_key)

In [40]:
# Selecting which sheet to pulling the data
sheet = workbook.worksheet("Sheet1")
# Pulling the data and transform it to the data frame
values = sheet.get_all_values()
flagcodes = pd.DataFrame(values[1:], columns=values[0])

In [42]:
flagcodes.columns = ["country", "code"]

In [92]:
table = us_china.merge(flagcodes, on="country", how="left")

In [93]:
# dw.create_chart(chart_type="tables", data=table)
table_id = "BWxT5"

In [94]:
table

,country,us,china,code
0,Cook Islands,72680,6.309566e+07,:ck:
1,Fiji,24253128,3.073780e+08,:fj:
2,"Micronesia, Federated States of",868669977,6.411541e+07,NaN
3,Niue,63382,2.000000e+04,:nu:
4,"Oceania, regional",80034401,2.000000e+05,NaN
5,Papua New Guinea,73120596,7.123071e+08,:pg:
6,Samoa,13492725,3.542694e+08,:ws:
7,Solomon Islands,29052925,3.868800e+04,:sb:
8,Tonga,14374286,2.176662e+08,:to:
9,Vanuatu,70973917,2.262781e+08,:vu:


In [95]:
df_mapping = pd.DataFrame(
    {
        "countries": [
            "Cook Islands",
            "Fiji",
            "Micronesia, Federated States of",
            "Niue",
            "Papua New Guinea",
            "Samoa",
            "Solomon Islands",
            "Tonga",
            "Vanuatu",
            "Oceania, regional",
            "Grand Total",
        ],
    }
)
sort_mapping = df_mapping.reset_index().set_index("countries")

In [96]:
table["num"] = table["country"].map(sort_mapping["index"])

In [97]:
table = table.sort_values("num")

In [98]:
table["country"] = table["code"] + " " + table["country"]

In [99]:
table = table.drop(columns=["code"])

In [100]:
dw.add_data(chart_id=table_id, data=table)

<Response [204]>

In [101]:
table

,country,us,china,num
0,:ck: Cook Islands,72680,6.309566e+07,0
1,:fj: Fiji,24253128,3.073780e+08,1
2,NaN,868669977,6.411541e+07,2
3,:nu: Niue,63382,2.000000e+04,3
5,:pg: Papua New Guinea,73120596,7.123071e+08,4
6,:ws: Samoa,13492725,3.542694e+08,5
7,:sb: Solomon Islands,29052925,3.868800e+04,6
8,:to: Tonga,14374286,2.176662e+08,7
9,:vu: Vanuatu,70973917,2.262781e+08,8
4,NaN,80034401,2.000000e+05,9


In [102]:
us_china

,country,us,china
0,Cook Islands,72680,6.309566e+07
1,Fiji,24253128,3.073780e+08
2,"Micronesia, Federated States of",868669977,6.411541e+07
3,Niue,63382,2.000000e+04
4,"Oceania, regional",80034401,2.000000e+05
5,Papua New Guinea,73120596,7.123071e+08
6,Samoa,13492725,3.542694e+08
7,Solomon Islands,29052925,3.868800e+04
8,Tonga,14374286,2.176662e+08
9,Vanuatu,70973917,2.262781e+08


### time series